# XGBoost Regression Example

In [ ]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name

sagemaker_session = sagemaker.Session()
base_job_prefix = 'xgboost-example'
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()
s3_prefix = base_job_prefix

training_instance_type = 'ml.m5.xlarge'

## Download Data and Prepare Training Input in S3

In [24]:
!aws s3 cp s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv .

download: s3://sagemaker-sample-files/datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv to ./abalone_dataset1_train.csv


In [25]:
!aws s3 cp abalone_dataset1_train.csv s3://{default_bucket}/xgboost-regression/train.csv

upload: ./abalone_dataset1_train.csv to s3://sagemaker-us-east-1-906815961619/xgboost-regression/train.csv


In [26]:
from sagemaker.inputs import TrainingInput
training_path = f's3://{default_bucket}/xgboost-regression/train.csv'
train_input = TrainingInput(training_path, content_type="text/csv")

## Retrieve XGBoost Image and Prepare Training Estimator W/ HyperParameters

In [27]:
model_path = f's3://{default_bucket}/{s3_prefix}/xgb_model'

image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role
)

xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

## Model Training

In [28]:
xgb_train.fit({'train': train_input})

2021-11-02 21:02:01 Starting - Starting the training job...
2021-11-02 21:02:24 Starting - Launching requested ML instancesProfilerReport-1635886920: InProgress
......
2021-11-02 21:03:24 Starting - Preparing the instances for training......
2021-11-02 21:04:25 Downloading - Downloading input data...
2021-11-02 21:04:45 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[21:05:09] 2923x8 matrix with 23384 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node training.
INFO:root:Train matri

## Retrieve Model Data

In [30]:
model_artifacts = xgb_train.model_data
model_artifacts

's3://sagemaker-us-east-1-906815961619/xgboost-example/xgb_model/sagemaker-xgboost-2021-11-02-21-02-00-815/output/model.tar.gz'

## Create SM Client to Create Model, EP Config, EP

In [31]:
sm_client = boto3.client(service_name='sagemaker')

## Model Creation

In [32]:
from time import gmtime, strftime
model_name = 'xgboost-reg' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Model name: ' + model_name)

reference_container = {
    "Image": image_uri,
    "ModelDataUrl": model_artifacts
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer= reference_container)

print("Model Arn: " + create_model_response['ModelArn'])

Model name: xgboost-reg2021-11-02-21-10-16
Model Arn: arn:aws:sagemaker:us-east-1:906815961619:model/xgboost-reg2021-11-02-21-10-16


## Endpoint Config Creation

In [33]:
endpoint_config_name = 'xgboost-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
instance_type='ml.c5d.18xlarge'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic',
        }])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: xgboost-config2021-11-02-21-10-40
Endpoint config Arn: arn:aws:sagemaker:us-east-1:906815961619:endpoint-config/xgboost-config2021-11-02-21-10-40


## Endpoint Creation

In [34]:
%%time

import time

endpoint_name = 'xgboost-reg' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: xgboost-reg2021-11-02-21-11-05
Endpoint Arn: arn:aws:sagemaker:us-east-1:906815961619:endpoint/xgboost-reg2021-11-02-21-11-05
Endpoint Status: Creating
Waiting for xgboost-reg2021-11-02-21-11-05 endpoint to be in service...
CPU times: user 97.3 ms, sys: 10.2 ms, total: 107 ms
Wall time: 3min 3s


## Sample Invocation

In [39]:
import boto3
smr = boto3.client('sagemaker-runtime')

resp = smr.invoke_endpoint(EndpointName=endpoint_name, Body=b'.345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0', 
                           ContentType='text/csv')

print(resp['Body'].read())

b'4.566554546356201'
